# Indian School of Business
## AMPBA - Term
## Group - 13
### MnCA Assignment — Part 1 : Product Optimization Exercise

### Group 13 - Member Details

| PGID     | Name               |
|----------|-------------------|
| 12420072 | Sanjukta Ghosh    |
| 12420042 | Mili Kapdia       |
| 12420055 | Abhinav Paul      |
| 12420007 | Bhaskar Yerramilli|

In [ ]:

import pandas as pd
import numpy as np

# Load preference parameters (full file)
prefs = pd.read_csv('mugs-preference-parameters-full.csv')
prefs.head()


,Cust,pPr30,pPr10,pPr05,pIn0.5,pIn1,pIn3,pCp12,pCp20,pCp32,...,pCnLk,pBrA,pBrB,pBrC,IPr,Iin,ICp,ICl,Icn,IBr
0,1,1,3,7,1,3,7,1,7,6,...,7,7,5,1,5,8,32,7,34,14
1,2,1,6,7,1,4,7,1,7,6,...,7,7,5,1,6,3,17,10,51,15
2,3,1,6,7,1,4,7,7,4,1,...,7,1,3,7,55,5,5,8,16,9
3,4,1,6,7,1,4,7,7,1,3,...,7,7,1,5,12,9,3,40,20,16
4,5,1,4,7,1,4,7,1,7,2,...,7,3,1,7,56,6,19,9,0,11


In [ ]:

# Examine columns and respondent count
prefs.columns.tolist(), prefs.shape


(['Cust',
  'pPr30',
  'pPr10',
  'pPr05',
  'pIn0.5',
  'pIn1',
  'pIn3',
  'pCp12',
  'pCp20',
  'pCp32',
  'pClD',
  'pClF',
  'pClE',
  'pCnSl',
  'pCnSp',
  'pCnLk',
  'pBrA',
  'pBrB',
  'pBrC',
  'IPr',
  'Iin',
  'ICp',
  'ICl',
  'Icn',
  'IBr'],
 (311, 25))

In [ ]:

# Load product lexical order (243 candidates)
lex = pd.read_csv('mugs-products-lexical-order.csv', header=None)
lex.columns = ['Index','Price','Time','Capacity','Clean','Contain']
lex.head()


,Index,Price,Time,Capacity,Clean,Contain
0,1,$30,0.5 hrs,12 oz,Difficult,Slosh resistant
1,2,$30,0.5 hrs,12 oz,Difficult,Spill resistant
2,3,$30,0.5 hrs,12 oz,Difficult,Leak resistant
3,4,$30,0.5 hrs,12 oz,Fair,Slosh resistant
4,5,$30,0.5 hrs,12 oz,Fair,Spill resistant


In [ ]:

# Define attribute-to-cost maps (example as used in class files)
price_map = {'$5':5, '$10':10, '$30':30}
time_cost = {'0.5 hrs':2.0, '1 hrs':3.0, '3 hrs':6.0}
cap_cost = {'12 oz':1.0, '20 oz':2.0, '32 oz':4.0}
clean_cost = {'Difficult':0.5, 'Fair':1.0, 'Easy':1.6}
contain_cost = {'Slosh resistant':0.5, 'Spill resistant':0.8, 'Leak resistant':1.0}

# Scaling constant (logit scale)
c = 0.0139


In [ ]:

# Define incumbent products
inc1 = {'Price':'$10','Time':'1 hrs','Capacity':'12 oz','Clean':'Difficult','Contain':'Slosh resistant','Brand':'A'}
inc2 = {'Price':'$10','Time':'1 hrs','Capacity':'20 oz','Clean':'Fair','Contain':'Spill resistant','Brand':'B'}


In [ ]:
# Utility computation function for any product
def utility_for_product(prod):
    clean_map = {'Difficult': 'D', 'Fair': 'F', 'Easy': 'E'}
    contain_map = {'Slosh resistant': 'Sl', 'Spill resistant': 'Sp', 'Leak resistant': 'Lk'}
    price_col_map = {'$30': 'pPr30', '$10': 'pPr10', '$5': 'pPr05'}
    u = (prefs['IBr'] +  # Brand intercept
         prefs['IPr'] +  # Price intercept
         prefs['Iin'] +  # Time intercept
         prefs['ICp'] +  # Capacity intercept
         prefs['ICl'] +  # Clean intercept
         prefs['Icn'] +  # Contain intercept
         prefs[price_col_map[prod['Price']]] +
         prefs['pIn' + prod['Time'].replace(' hrs', '')] +
         prefs['pCp' + prod['Capacity'].replace(' oz', '')] +
         prefs['pCl' + clean_map[prod['Clean']]] +
         prefs['pCn' + contain_map[prod['Contain']]])
    return u

In [ ]:
# Compute utilities, logit shares, cost, margin, EPPP for 243 candidates
rows = []
for i, r in lex.iterrows():
    prod = {k: str(r[k]).strip() for k in ['Price','Time','Capacity','Clean','Contain']}
    prod['Brand'] = 'C'
    u_inc1 = utility_for_product(inc1)
    u_inc2 = utility_for_product(inc2)
    u_cand = utility_for_product(prod)
    exp_inc1 = np.exp(c * u_inc1)
    exp_inc2 = np.exp(c * u_inc2)
    exp_cand = np.exp(c * u_cand)
    denom = exp_inc1 + exp_inc2 + exp_cand
    share = (exp_cand / denom).mean()
    tkey, ckey, clkey, conkey = prod['Time'], prod['Capacity'], prod['Clean'], prod['Contain']
    cost = time_cost[tkey] + cap_cost[ckey] + clean_cost[clkey] + contain_cost[conkey]
    margin = price_map[prod['Price']] - cost
    eppp = share * margin
    rows.append({'index':int(r['Index']), **prod, 'share':share, 'cost':cost, 'margin':margin, 'eppp':eppp})

table = pd.DataFrame(rows).sort_values('index').reset_index(drop=True)
display(table.head())

,index,Price,Time,Capacity,Clean,Contain,Brand,share,cost,margin,eppp
0,1,$30,0.5 hrs,12 oz,Difficult,Slosh resistant,C,0.297362,4.0,26.0,7.731422
1,2,$30,0.5 hrs,12 oz,Difficult,Spill resistant,C,0.306003,4.3,25.7,7.864286
2,3,$30,0.5 hrs,12 oz,Difficult,Leak resistant,C,0.315069,4.5,25.5,8.034264
3,4,$30,0.5 hrs,12 oz,Fair,Slosh resistant,C,0.308930,4.5,25.5,7.877727
4,5,$30,0.5 hrs,12 oz,Fair,Spill resistant,C,0.317757,4.8,25.2,8.007486


In [ ]:

# Candidate 45 (the proposed one)
cand45 = table.loc[table['index']==45]
cand45


,index,Price,Time,Capacity,Clean,Contain,Brand,share,cost,margin,eppp
44,45,$30,1 hrs,20 oz,Easy,Leak resistant,C,0.349656,7.6,22.4,7.832299


In [ ]:

# Identify optimal and business-choice products
table['price_val'] = table['Price'].map(price_map)
table['revenue_per_person'] = table['share'] * table['price_val']

best_eppp = table.loc[table['eppp'].idxmax()]
best_share = table.loc[table['share'].idxmax()]
best_margin = table.loc[table['margin'].idxmax()]
best_cost = table.loc[table['cost'].idxmin()]
best_revenue = table.loc[table['revenue_per_person'].idxmax()]

best_eppp, best_share, best_margin, best_cost, best_revenue


(index                              6
 Price                            $30
 Time                         0.5 hrs
 Capacity                       12 oz
 Clean                           Fair
 Contain               Leak resistant
 Brand                              C
 share                       0.327005
 cost                             5.0
 margin                          25.0
 eppp                        8.175137
 price_val                         30
 revenue_per_person          9.810165
 Name: 5, dtype: object,
 index                            234
 Price                             $5
 Time                           3 hrs
 Capacity                       20 oz
 Clean                           Easy
 Contain               Leak resistant
 Brand                              C
 share                       0.378479
 cost                            10.6
 margin                          -5.6
 eppp                        -2.11948
 price_val                          5
 revenue_per_person      

In [ ]:

# Save final table
table.to_csv('mugs-243-table.csv', index=False)
print('Saved mugs-243-table.csv')


Saved mugs-243-table.csv
